# <b><span style='color:#0B2F9F'>Pengantar</span></b>

Dalam pengelolaan data, kemampuan untuk menampilkan informasi dengan format yang tepat serta menggali data secara mendalam menjadi hal yang sangat penting. Format angka dan tanggal membantu memastikan data tersaji secara konsisten, mudah dibaca, dan sesuai konteks, sehingga hasil analisis lebih akurat dan informatif. Sementara itu, penggunaan subquery memungkinkan pengambilan data secara lebih fleksibel dan efisien, dengan cara memecah proses pencarian menjadi langkah-langkah yang lebih terarah. Dengan memahami kedua konsep ini, seseorang dapat mengolah data dengan lebih rapi, melakukan analisis yang lebih mendalam, serta menghasilkan informasi yang lebih bermakna dan dapat diandalkan.


# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

In [ ]:
# Buat BigQuery client
project_id = 'dqlab-475713'
client = bigquery.Client(project = project_id)

Sebagai Data Engineer, Antara (Jr. Data Engineer di DQPizza) ditugaskan untuk membuat data mart untuk kebutuhan detail transaksi. Dengan memanfaatkan relasi antar table, Antara mengeksekusi suatu perintah  untuk membuat table f_detail_transaksi (sebuah tabel fact yang berisi data transaksi secara rinci, mencakup ukuran-ukuran kuantitatif seperti jumlah, harga, total nilai transaksi, dan lain sebagainya) hingga terbentuklah tabel f_detail_transaksi.

<br>
<code>SELECT * FROM dqlab-9876543.dq_pizza.f_detail_transaksi</code>
<br><br>
<img src="https://raw.githubusercontent.com/bachtiyarma/Material/main/Image/Materi-SQL/SQL%20-%20f_detail_transaksi.png" width="30%">

Data dibawah (f_detail_transaksi) akan digunakan pada modul ini

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

# <b>A. <span style='color:#0B2F9F'><code>Data Type</code></span></b>

Data type dalam BigQuery adalah cara untuk mendefinisikan jenis data yang akan disimpan dalam tabel. Setiap kolom dalam tabel memiliki tipe data tertentu, yang menentukan apa yang dapat disimpan di kolom tersebut dan bagaimana data tersebut dapat diproses. Memahami tipe data ini penting untuk desain skema yang efisien dan untuk memastikan bahwa kueri dapat dijalankan dengan optimal.

<br>
<img src="https://hackolade.com/help/lib/BigQuery%20data%20types%20table.png" width="40%">

docs : <i><a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types">https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types</a></i>

# <b>B. <span style='color:#0B2F9F'><code>Operasi Tipe Data Numerik</code></span></b>

Dalam SQL, tipe data numerik digunakan untuk menyimpan nilai angka dan dapat dikelompokkan menjadi beberapa jenis sesuai peruntukannya (INT, FLOAT, NUMERIC, dll). Selain itu pada tipe data Numerik juga dapat dilakukan operasi matematis diantaranya :
<ol>
    <li>Operasi Aritmatika</li>
    <li>Fungsi Matematis</li>
    <li>Agregasi</li>
</ol>

## <b>B.1. <span style='color:#0B2F9F'><code>Operasi Aritmatika</code></span></b>
SQL mendukung operasi dasar aritmatika seperti penjumlahan, pengurangan, perkalian, pembagian dan modulo.

<img src="https://mazamanbd.wordpress.com/wp-content/uploads/2014/09/dd.jpg"><br><br>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL%20-%20OperasiAritmatika.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/operators">https://cloud.google.com/bigquery/docs/reference/standard-sql/operators</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel f_detail_transaksi, tampilkan semua kolom dan tambahkan satu kolom 'total_price' yang merupakan hasil perkalian antara harga dengan kuantitas (terbeli)! Urutkan berdasarkan kuantitas paling besar hingga ke terkecil </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  (price * quantity) AS total_price
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC;

#### <i><b><span style='color:#55679C'>Quest</span> : Selain total price, tambahkan juga total profit yang didapat tiap transaksi! (Lanjutkan hasil query quest diatas) </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT *,
(price * quantity) AS total_price,
((price - production_cost) * quantity) AS total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
ORDER BY quantity DESC;

#### <i><b><span style='color:#55679C'>Quest</span> : Bagaimana dengan customer paling loyal? Paling loyal diukur dari banyak profit restaurant yang dihasilkan. Tampilkan customer_id, total_frequency (berapa kali pembelian dalam setahun) dan total_profit yang diperoleh perusahaan, kelompokkan tiap customer_id! Urutkan berdasarkan customer_id dengan profit paling tinggi ke paling rendah! </b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

Select customer_id,
       COUNT(order_id) as total_frequency,
       SUM((price-production_cost) * quantity) as total_profit
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY customer_id
order by total_profit desc

## <b>B.2. <span style='color:#0B2F9F'><code>Fungsi Matematis</code></span></b>
SQL juga mendukung fungsi matematis dasar yang disajikan pada tabel berikut :

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>ABS()</code></td>
            <td>Mengambil nilai absolut</td>
        </tr>
        <tr>
            <td><code>ROUND()</code></td>
            <td>Membulatkan angka</td>
        </tr>
        <tr>
            <td><code>CEIL()</code></td>
            <td>Membulatkan ke atas</td>
        </tr>
        <tr>
            <td><code>FLOOR()</code></td>
            <td>Membulatkan ke bawah</td>
        </tr>
        <tr>
            <td><code>POW()</code></td>
            <td>Menghitung pangkat</td>
        </tr>
    </tbody>
</table>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL-FungsiMatematis.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Sendja (Data Analyst di DQPizza) baru menyadari bahwa profit yang ia hitung pada quest sebelumnya merupakan profit sebelum pajak. Asumsikan pajak yang perlu dikeluarkan adalah 11% tiap kali transaksi. Hitung profit_before_tax, profit_after_tax dan tax_paid tiap order_id dan customer_id! Lakukan perhitungan untuk setiap transaksi yang valid (order_details_id tidak kosong)!</b></i>

Note : Bulatkan profit after tax sebanyak 2 angka dibelakang koma agar lebih rapi

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
order_id,
customer_id,
ROUND(SUM((price - production_cost) * quantity),2) AS profit_before_tax,
ROUND(SUM((price - production_cost) * quantity)*(1-11/100),2) AS profit_after_tax,
SUM(((price - production_cost) * quantity) - ((price - production_cost) * quantity)*(1-11/100)) AS tax_paid
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
where order_details_id IS NOT NULL
GROUP BY order_id, customer_id

# <b>C. <span style='color:#0B2F9F'><code>Operasi Tipe Data Date</code></span></b>

Dalam SQL, tipe data date digunakan untuk menyimpan atau mengoperasikan data tanggal. Tipe data tanggal dapat dilakukan operasi berikut, diantaranya :
<ol>
    <li>Menampilkan Tanggal & Waktu saat ini</li>
    <li>Menambahkan atau Mengurangi Tanggal</li>
    <li>Mendapatkan Bagian Tanggal</li>
    <li>Menghitung Selisih Tanggal</li>
</ol>

<br><b>NOTE</b><br><br>
Hal-hal yang perlu diperhatikan dalam penggunaan operasi `date` adalah :
<ul>
    <li>Format tanggal standar SQL adalah 'YYYY-MM-DD' atau tahun - bulan - tanggal, contoh : 2024-10-14 menunjukan tanggal 14 October 2024</li>
    <li>Mengakses tanggal pada SQL perlu menggunakan tanda apit petik satu, contoh : '2024-10-14'</li>
    <li>Pastikan nilai atau kolom tanggal tipe datanya adalah DATE atau TIMESTAMP, jika tidak maka bisa dikonversi menjadi <code>DATE(tanggal)</code> atau <code>TIMESTAMP(tanggal)</code></li>
</ul>

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions</a></i>

## <b>C.1. <span style='color:#0B2F9F'><code>Menampilkan Tanggal & Waktu Saat Ini</code></span></b>

Untuk menampilkan tanggal dan waktu saat ini dapat menggunakan fungsi berikut
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>CURRENT_DATE()</code></td>
            <td>Mendapatkan Tanggal Hari ini Saja</td>
        </tr>
        <tr>
            <td><code>CURRENT_TIMESTAMP()</code></td>
            <td>Mendapatkan Tanggal & Waktu Hari ini (UTC)</td>
        </tr>
    </tbody>
</table>

UTC atau (Universal Time Coordinated) adalah waktu dasar yang tidak terpengaruh oleh zona waktu dan <i>daylight saving time</i>. UTC digunakan sebagai acuan untuk menghitung waktu di berbagai zona waktu di seluruh dunia (standar Greenwich Mean Time (London)). Selengkapnya baca <a href="https://internasional.kompas.com/read/2021/03/04/192042570/sejarah-penetapan-zona-waktu-di-dunia-hingga-usulan-penghapusannya?page=all">disini</a>.

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan semua kolom pada tabel f_detail_transaksi dan tambahkan kolom data_collection_time sebagai kolom metadata waktu pengambilan data</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *,
  TIMESTAMP(DATETIME(CURRENT_TIMESTAMP(), "Asia/Jakarta")) AS data_collection_time
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

## <b>C.2. <span style='color:#0B2F9F'><code>Mengubah String menjadi Tanggal</code></span></b>

Adakalanya format tanggal yang disimpan pada database tidak disimpan dalam format date yang baku (<a href="https://www.iso.org/iso-8601-date-and-time-format.html">ISO 8601</a>) sehingga perlu penyesuaian. Gunakan : <br><br>
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>PARSE_DATE(format, string_expression)</code></td>
            <td>Mengubah String menjadi Date</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE(format, date_expression)</code></td>
            <td>Mengubah Date menjadi String</td>
        </tr>
    </tbody>
</table><br>

Untuk detail format tanggal akses link berikut https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/format-elements

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan order_date dan order_date_iso pada tabel f_detail_transaksi. Dimana kolom order_date merupakan kolom dengan data asli dan order_date_iso merupakan kolom order_date yang diubah formatnya menjadi YYYY-MM-DD sesuai ISO 8601!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  order_date,
  FORMAT_DATE('%Y-%m-%d', order_date) AS order_date_iso
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

## <b>C.3. <span style='color:#0B2F9F'><code>Menambahkan atau Mengurangi Tanggal</code></span></b>

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>DATE_ADD(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Menambahkan Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_SUB(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Mengurangi Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_DIFF(end_date, start_date, date_part)</code></td>
            <td>Menghitung selisih tanggal</td>
        </tr>
    </tbody>
</table><br>

Sebagai contoh misalkan hari ini tanggal 30 Oktober 2025. Ingin diperoleh tanggal tepat 3 bulan yang lalu maka dapat digunakan sintaks `DATE_SUB(DATE('2025-10-30'), INTERVAL 3 MONTH)` maka akan diperoleh tanggal 30 Juli 2025.



#### <i><b><span style='color:#55679C'>Quest</span> : Manajer ingin melakukan analisa demografi customer, pada tabel customers tampilkan customer_id, customer_name, gender dan age-nya per hari ini!</b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT * FROM `dqlab-9876543.dq_pizza.customers`

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  gender,
  birth_date,
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) AS age
FROM `dqlab-9876543.dq_pizza.customers`


## <b>C.5. <span style='color:#0B2F9F'><code>Mendapatkan Bagian pada Tanggal</code></span></b>

Mengambil bagian tertentu dari tipe data tanggal, waktu, atau timestamp terkadang sangat perlu dilakukan untuk analisa data. Jika ingin didapatkan elemen tertentu, seperti tahun, bulan, hari, atau jam, dari nilai tanggal atau waktu gunakan fungsi berikut<br><br>

<code>EXTRACT(bagian FROM tanggal)</code>
<br><br>

Bagian yang bisa diambil pada tanggal diantaranya adalah
<table border="1">
    <thead>
        <tr>
            <th>Bagian</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>YEAR</code></td>
            <td>Mendapatkan Tahun pada Tanggal</td>
        </tr>
        <tr>
            <td><code>MONTH</code></td>
            <td>Mendapatkan Bulan pada Tanggal</td>
        </tr>
        <tr>
            <td><code>DAY</code></td>
            <td>Mendapatkan Hari (1-31) pada Tanggal</td>
        </tr>
        <tr>
            <td><code>WEEK</code></td>
            <td>Mendapatkan Minggu ke- pada Tanggal</td>
        </tr>
         <tr>
            <td><code>QUARTER</code></td>
            <td>Mendapatkan Kuartal pada Tanggal</td>
        </tr>
        <tr>
            <td><code>HOUR</code></td>
            <td>Mendapatkan Jam pada Waktu</td>
        </tr>
        <tr>
            <td><code>MINUTE</code></td>
            <td>Mendapatkan Menit pada Waktu</td>
        </tr>
        <tr>
            <td><code>SECOND</code></td>
            <td>Mendapatkan Detik pada Waktu</td>
        </tr>
    </tbody>
</table>
<br>

Namun jika menginginkan bagian yang lebih detail seperti Nama Bulan atau Nama Hari pada tanggal tertentu atau menampilkan format tanggal sesuai dengan keinginan dapat menggunakan sintaks `FORMAT_DATE(format_element, tanggal)`. <br><br>

<table border="1">
    <thead>
        <tr>
            <th>Sintaks</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>FORMAT_DATE('%A', tanggal)</code></td>
            <td>Mendapatkan Nama Hari pada Tanggal Tertentu</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE('%B', tanggal)</code></td>
            <td>Mendapatkan Nama Bulan pada Tanggal Tertentu</td>
        </tr>
    </tbody>
</table>
<br>

Detail format_element dapat diakses dilaman <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/format-elements#format_elements_date_time"> berikut</a>.





#### <i><b><span style='color:#55679C'>Quest</span> : Hitung total transaksi dan total pendapatan yang diperoleh tiap bulan pada tahun 2024 di DQPizza! </b></i><br>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  EXTRACT(YEAR FROM PARSE_DATE('%m/%d/%Y', order_date)) AS year,
  EXTRACT(MONTH FROM PARSE_DATE('%m/%d/%Y', order_date)) AS month,
  FORMAT_DATE('%B', PARSE_DATE('%m/%d/%Y', order_date)) AS month_name,
  COUNT(DISTINCT order_id) AS total_transaksi,
  SUM(price * quantity) AS total_pendapatan
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
GROUP BY
  1, 2, 3
ORDER BY
  1, 2

# <b>D. <span style='color:#0B2F9F'><code>SUB-QUERY</code></span></b>
Subquery adalah sebuah query (pernyataan SQL) yang terletak di dalam query lain. Subquery dapat digunakan untuk mengambil data yang kemudian akan digunakan oleh query utama.<br><br>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, hitung rata-rata umur customer dan bulatkan ke dua tempat desimal!</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) AS age
FROM `dqlab-9876543.dq_pizza.customers`
WHERE
  DATE_DIFF(CURRENT_DATE(), birth_date, YEAR) > (SELECT AVG(DATE_DIFF(CURRENT_DATE(), birth_date, YEAR)) AS age FROM `dqlab-9876543.dq_pizza.customers`)

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, tampilkan customer_id, age dan gendernya dimana umur customer tersebut lebih dari rata-rata umur customer secara keseluruhan!</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

WITH temp_tbl AS (
  SELECT
    order_id,
    customer_id,
    ROUND(SUM(((price - production_cost) * quantity)), 2) AS profit_before_tax
  FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
  WHERE order_details_id IS NOT NULL
  GROUP BY
    order_id, customer_id
)
  SELECT
    order_id,
    customer_id,
    profit_before_tax,
    (1-0.11) * profit_before_tax AS profit_after_tax,
    0.11 * profit_before_tax AS tax_paid
  FROM temp_tbl

# <b>E. <span style='color:#0B2F9F'><code>Common Table Expression (CTE)</code></span></b>
CTE, atau Common Table Expression, adalah fitur dalam SQL yang memungkinkan Anda untuk mendefinisikan sebuah query sementara yang dapat digunakan dalam query lain. CTE membantu dalam membuat query lebih mudah dibaca dan dipelihara, serta memungkinkan penggunaan rekursi dalam query.

<img src="https://learnsql.com/blog/sql-common-table-expression-guide/cte_syntax_example.webp" width="50%">

#### <i><b><span style='color:#55679C'>Quest</span> : Siapakah customer yang paling banyak spending money di DQPizza?</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

WITH temp_tbl AS (
  SELECT
    customer_id,
    ROUND(SUM(((price - production_cost) * quantity)), 2) AS profit_before_tax
  FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`
  WHERE order_details_id IS NOT NULL
  GROUP BY
    order_id, customer_id
)
  SELECT
    customer_id,
    profit_before_tax
  FROM temp_tbl
  WHERE profit_before_tax = (SELECT MAX(profit_before_tax) FROM temp_tbl)

Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/muhamad-rasyid-aditya-17637a225/"><img src="https://img.shields.io/badge/-© 2025 Muhamad Rasyid Aditya-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
